In [ ]:
import rebound as rb
import reboundx as rbx
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sim = rb.Simulation()
sim.units = ('yr', 'AU', 'Msun')
sim.add(m = 1)
sim.add(m = 1.e-4, a=0.5, e=0, inc = 0)
sim.move_to_com()

In [ ]:
simx = rbx.Extras(sim)
ide = simx.load_force("inner_disk_edge")
simx.add_force(ide)

ide.params["inner_disk_edge_position"] = 0.1
ide.params["disk_edge_width"] = 0.03*(0.1**0.25)                                                    #hedge = 0.03 x (dedge^0.25)
ps = sim.particles

In [ ]:
sim.integrator = 'whfast'
sim.dt = np.sqrt(0.1**3)/20

In [ ]:
times = np.linspace(0, 1e4, 1000)
a_integration = np.zeros((1000))
for i, t in enumerate(times):
    sim.integrate(t)
    a_integration[i] = ps[1].a


a_analytical = np.exp(-(times)/(2e4))

In [ ]:
plt.plot(times*0.001, a_integration, label = 'Numerical evolution', c = 'green') 
plt.plot(times*0.001, a_analytical, label = 'Analytical evolution', c = 'brown')
plt.xlim(np.min(times)*0.001, np.max(times)*0.001) 
plt.xlabel('time [kyr]')
plt.ylabel('Semi-major axis [AU]')
plt.legend()